In [1]:
%run ../src/ipyautoui/__init__.py
%load_ext lab_black

In [9]:
" not yet implemented".upper()

' NOT YET IMPLEMENTED'

# Validation - NOTE:  NOT YET IMPLEMENTED

- jsonschema validation occurs on-change by default (TODO: implement!)
    - review https://github.com/pydantic/pydantic-core
- the top-level pydantic model can also be validated on request (slows things down...?)
    - possible to implement recursive validation with pydantic? 

In [2]:
from jsonschema import validate, Draft202012Validator, ErrorTree

# A sample schema, like what we'd get from json.load()
schema = {
    "type" : "object",
    "properties" : {
        "price" : {"type" : "number"},
        "name" : {"type" : "string"},
    },
}

# If no exception is raised by validate(), the instance is valid.
validate(instance={"name" : "Eggs", "price" : 34.99}, schema=schema)

validate(
    instance={"name" : "Eggs", "price" : "Invalid"}, schema=schema,
)      

ValidationError: 'Invalid' is not of type 'number'

Failed validating 'type' in schema['properties']['price']:
    {'type': 'number'}

On instance['price']:
    'Invalid'

In [3]:
schema = {
    "items": {
        "anyOf": [{"type": "string", "maxLength": 2}, {"type": "integer", "minimum": 5}]
    }
}
instance = [{}, 3, "foo"]
v = Draft202012Validator(schema)
errors = sorted(v.iter_errors(instance), key=lambda e: e.path)

In [4]:
from ipyautoui.constants import TRUE_BUTTON_KWARGS, BUTTON_WIDTH_MIN, BUTTON_HEIGHT_MIN
import ipyw as w

BLANK_BUTTON_KWARGS = dict(
    icon="times",
    style={"button_color": "red"},
    layout={"width": BUTTON_WIDTH_MIN, "height": BUTTON_HEIGHT_MIN},
    disabled=True,
)

BLANK_BUTTON_KWARGS = dict(
    icon="check",
    style={"button_color": "white"},
    layout={"width": BUTTON_WIDTH_MIN, "height": BUTTON_HEIGHT_MIN},
    disabled=True,
)
w.HBox([w.Button(**BLANK_BUTTON_KWARGS), w.Text("asdfasdf")])

In [5]:
for error in errors:
    for suberror in sorted(error.context, key=lambda e: e.schema_path):
        print(list(suberror.schema_path), suberror.message, sep=", ")

[0, 'type'], {} is not of type 'string'
[1, 'type'], {} is not of type 'integer'
[0, 'type'], 3 is not of type 'string'
[1, 'minimum'], 3 is less than the minimum of 5
[0, 'maxLength'], 'foo' is too long
[1, 'type'], 'foo' is not of type 'integer'


In [6]:
schema = {
    "type": "array",
    "items": {"type": "number", "enum": [1, 2, 3]},
    "minItems": 3,
}
instance = ["spam", 2]

v = Draft202012Validator(schema)
for error in sorted(v.iter_errors(["spam", 2]), key=str):
    print(error.message)

'spam' is not of type 'number'
'spam' is not one of [1, 2, 3]
['spam', 2] is too short


In [7]:
tree = ErrorTree(v.iter_errors(instance))

In [8]:
from pydantic import BaseModel, ValidationError, root_validator


class UserModel(BaseModel):
    username: str
    password1: str
    password2: str

    @root_validator(pre=True)
    def check_card_number_omitted(cls, values):
        assert "card_number" not in values, "card_number should not be included"
        return values

    @root_validator
    def check_passwords_match(cls, values):
        pw1, pw2 = values.get("password1"), values.get("password2")
        if pw1 is not None and pw2 is not None and pw1 != pw2:
            raise ValueError("passwords do not match")
        return values


print(UserModel(username="scolvin", password1="zxcvbn", password2="zxcvbn"))
# > username='scolvin' password1='zxcvbn' password2='zxcvbn'
try:
    UserModel(username="scolvin", password1="zxcvbn", password2="zxcvbn2")
except ValidationError as e:
    print(e)

username='scolvin' password1='zxcvbn' password2='zxcvbn'
1 validation error for UserModel
__root__
  passwords do not match (type=value_error)


In [33]:
import stringcase as sc

name = w.Text()
name_ = w.Text(disabled=True)
vbox = w.VBox(
    [
        w.HBox([name, w.Label("schedule name in plain english (with spaces)")]),
        w.HBox([name_, w.Label("schedule name for machines")]),
    ]
)


def on_name_change(on_change):
    name_.value = sc.pascalcase(sc.snakecase(name.value.lower())).strip("_")


name.observe(on_name_change, "value")

vbox